<a href="https://colab.research.google.com/github/see-bear/public/blob/main/ConnectedIntelligence_2_CF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# cell 1 install Gradio
!pip install gradio
# Import Libraries
import pandas as pd
import logging


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [3]:
#Cell 2 load paradigm array
# @title Default title text
# Load the paradigm array from the uploaded Excel file
paradigm_file_path =  '/content/Class_A_array3.xlsx'
paradigm_df = pd.read_excel(paradigm_file_path)
paradigm_array = paradigm_df.values.flatten()  # Convert to a flat array
print("Paradigm Array:", paradigm_array)

Paradigm Array: [nan '224YYYY' '2242023 - Client Project Name Sample' nan nan nan nan nan
 nan nan nan 'Level 2 Matrix' 'Level 2 Category name' 'Subfolder 1'
 'Subfolder 2' 'Subfolder 3' 'Subfolder 4' 'Subfolder 5' 'Subfolder 6'
 'Subfolder 7' 'Subfolder 8' 'Subfolder 9' nan 'ADM' 'AGRMT' 'CHANGE'
 'COST' 'PLAN' nan nan nan nan nan nan 'COR' 'MECH' 'ELECT' 'ELEV' 'ICAT'
 'PM' nan nan nan nan nan 'DWG' 'ARCH' 'MECH' 'ELEC' 'ZZZZ' nan nan nan
 nan nan nan 'DSGN' 'CALC' 'MANFR' 'MTGM' 'NOTE' 'RPT' nan nan nan nan nan
 'INSP' 'SI' 'NOC' 'CD' 'CHOR' 'ICOR' 'PRGD' 'PAMT' 'RPTS' 'SDWG' nan
 'PUB' 'REV' 'TENDER' 'ADD' 'IFC' 'CIM' 'CIE' 'CIC' 'OM' nan
 'Level 3 Matrix: COR Subfolders' 'Level 3 category name ' 'Subfolder 1'
 'Subfolder 2' 'Subfolder 3' 'Subfolder 4' 'Subfolder 5' nan nan nan nan
 nan 'COR' 'OWNER' 'DESTM' 'CONTRCTR' 'SUBCONTR' 'AUTHRTY' nan nan nan nan
 nan 'MECH' 'OWNER' 'DESTM' 'CONTRCTR' 'SUBCONTR' 'AUTHRTY' nan nan nan
 nan nan 'ELECT' 'OWNER' 'DESTM' 'CONTRCTR' 'SUBCONTR' '

In [4]:
# Cell 3 investigate folder structure
import os

def investigate_folder_structure(path='content/224_Sample.zip'):
    folder_structure = []
    # Initialize root and level outside the loop to store their final values
    root = ''
    level = 0

    for root, dirs, _ in os.walk(path):  # Assign root and level in the loop
        for dir_name in dirs:
            level = root.replace(path, '').count(os.sep) # use path instead of root_path
            folder_structure.append((f"L{level}", dir_name))

    # Now root and level will have the last values from the loop
    return folder_structure, root, level # Return root and level

# Call the function and assign the returned values to variables
folder_structure, root_val, level_val = investigate_folder_structure()
print(f"Root: {root_val}, Level: {level_val}")

Root: , Level: 0


In [5]:
#cell 4 setup temp directory
import os
import zipfile
import shutil
import gradio as gr

def compare_folder_structures(zip_file):
    # Define a temporary directory to extract the zip file
    temp_dir = "temp_dir"
    try:
        # Create the temporary directory
        os.makedirs(temp_dir, exist_ok=True)

        # Try to open and extract the zip file
        try:
            with zipfile.ZipFile(zip_file.name, 'r') as z:
                z.extractall(temp_dir)
        except zipfile.BadZipFile:
            return "Error: The uploaded file is not a valid zip file.", [], []

        # Investigate the folder structure with full paths
        folder_structure = []
        for root, dirs, _ in os.walk(temp_dir):
            for dir_name in dirs:
                # Use os.path.relpath for determining level
                level = os.path.relpath(root, temp_dir).count(os.sep)
                full_path = os.path.join(root, dir_name)
                folder_structure.append((f"L{level}", full_path))

        # Compare with the paradigm array (ensuring it contains only folder names)
        folder_structure_set = {os.path.basename(name) for _, name in folder_structure}
        missing_folders = [name for name in paradigm_array if name not in folder_structure_set]

        # Create the result table and missing folders table
        result = "Standards Compliant" if not missing_folders else "Lacks the following folders"
        result_table = [{"Level": level, "Name": os.path.basename(name), "Full Path": name} for level, name in folder_structure]
        missing_table = [{"Missing Folder": name} for name in missing_folders]

    except Exception as e:
        # Generic error handling for unexpected issues
        return f"Error: An unexpected issue occurred - {str(e)}", [], []
    finally:
        # Ensure temporary directory is cleaned up
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

    return result, result_table, missing_table, paradigm_array, "Debug logs: All steps completed successfully."


In [6]:
#cell 5 function to handle comparison of file structures
import os
import zipfile
import shutil
import pandas as pd

# Example paradigm array (replace this with actual data from your Excel file)
paradigm_array = ["Folder1", "Folder2", "Folder3/Subfolder"]

# Function to handle file structure comparison
def handle_user_actions(zip_file):
    temp_dir = "temp_dir"
    try:
        # Step 1: Validate the uploaded file
        if not zip_file.name.endswith(".zip"):
            return "Error: Uploaded file is not a .zip archive.", [], [], "Invalid input. Please upload a valid .zip file."

        # Step 2: Extract the zip file into a temporary directory
        os.makedirs(temp_dir, exist_ok=True)
        try:
            with zipfile.ZipFile(zip_file.name, 'r') as z:
                z.extractall(temp_dir)
        except zipfile.BadZipFile:
            return "Error: Uploaded file is not a valid zip file.", [], [], "Upload a valid .zip file."

        # Step 3: Investigate folder structure
        folder_structure = []
        for root, dirs, _ in os.walk(temp_dir):
            for dir_name in dirs:
                level = os.path.relpath(root, temp_dir).count(os.sep)
                folder_structure.append((f"L{level}", os.path.join(root, dir_name)))

        # Step 4: Validate data and compare with the paradigm array
        folder_names_set = {os.path.basename(name) for _, name in folder_structure}
        missing_folders = [folder for folder in paradigm_array if folder not in folder_names_set]

        # Step 5: Prepare outputs
        result = "Standards Compliant" if not missing_folders else "Lacks the following folders"
        result_table = [{"Level": level, "Name": os.path.basename(name)} for level, name in folder_structure]
        missing_table = [{"Missing Folder": folder} for folder in missing_folders]

        # Return results
        return result, result_table, missing_table, "Check missing folders and click 'Proceed' or 'Cancel'."

    except Exception as e:
        # Generic error handling
        return f"Error: {str(e)}", [], [], "An unexpected error occurred."

    finally:
        # Step 6: Ensure temp_dir cleanup
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

# Function to create missing folders based on user selection
def create_missing_folders(zip_file, selected_folders):
    temp_dir = "temp_dir"
    try:
        # Extract the zip file again
        os.makedirs(temp_dir, exist_ok=True)
        with zipfile.ZipFile(zip_file.name, 'r') as z:
            z.extractall(temp_dir)

        # Create missing folders
        created = []
        for folder in selected_folders:
            full_path = os.path.join(temp_dir, folder)
            os.makedirs(full_path, exist_ok=True)
            created.append(folder)

        return f"Successfully created folders: {', '.join(created)}"

    except Exception as e:
        return f"Error: {str(e)}"

    finally:
        # Cleanup temp_dir
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)


In [7]:
#cell 6 handle user actions
import gradio as gr
import os
import zipfile
import shutil
import pandas as pd

# Example paradigm array (replace this with actual data from your Excel file)
paradigm_array = ["Folder1", "Folder2", "Folder3/Subfolder"]
print("Paradigm Array:", paradigm_array)

# Function to handle file structure comparison
def handle_user_actions(zip_file):
    print(f"Uploaded file name: {zip_file.name}")  # Debugging line
    temp_dir = "temp_dir"
    try:
        print("Starting to process uploaded file...")
        print("Step 1: Starting file validation...")
        # Step 1: Validate the uploaded file
        if not zip_file.name.endswith(".zip"):
            print("Error: Uploaded file is not a zip file.")
            return "Error: Uploaded file is not a .zip archive.", [], [], "Invalid input. Please upload a valid .zip file."
        print("Step 1: File validation complete.")
        print("Step 2: Extracting file...")

        # Step 2: Extract the zip file into a temporary directory
        os.makedirs(temp_dir, exist_ok=True)
        try:
            with zipfile.ZipFile(zip_file.name, 'r') as z:
                print("Extracting files...")
                z.extractall(temp_dir)
        except zipfile.BadZipFile:
            print(f"Error during extraction: {e}")
            return "Error: Uploaded file is not a valid zip file.", [], [], "Upload a valid .zip file."
        print("Step 2: File extraction complete.")

        print("Step 3: Investigating folder structure...")
        # Step 3: Investigate folder structure
        print("Investigating folder structure...")
        folder_structure = []
        for root, dirs, _ in os.walk(temp_dir):
            for dir_name in dirs:
                level = os.path.relpath(root, temp_dir).count(os.sep)
                folder_structure.append((f"L{level}", os.path.join(root, dir_name)))
                print(f"Level {level}: {dir_name}")

        print("Step 4: Comparing folder structure...")
        # Step 4: Validate data and compare with the paradigm array
        print("Comparing with paradigm array...")
        folder_names_set = {os.path.basename(name) for _, name in folder_structure}
        missing_folders = [folder for folder in paradigm_array if folder not in folder_names_set]
        print("Step 4: Comparison complete.")
        print(f"Missing Folders: {missing_folders}")

        print("Step 5: Preparing outputs...")
        # Step 5: Prepare outputs
        result = "Standards Compliant" if not missing_folders else "Lacks the following folders"
        result_table = [{"Level": level, "Name": os.path.basename(name)} for level, name in folder_structure]
        missing_table = [{"Missing Folder": folder} for folder in missing_folders]

        # Return results
        return result, result_table, missing_table, "Check missing folders and click 'Proceed' or 'Cancel'."

    except Exception as e:
        print(f"Unhandled exception: {e}")
        # Generic error handling
        return f"Error: {str(e)}", [], [], "An unexpected error occurred."

    finally:
        # Step 6: Ensure temp_dir cleanup
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

# Function to create missing folders based on user selection
def create_missing_folders(zip_file, selected_folders):
    temp_dir = "temp_dir"
    try:
        # Extract the zip file again
        os.makedirs(temp_dir, exist_ok=True)
        with zipfile.ZipFile(zip_file.name, 'r') as z:
            z.extractall(temp_dir)

        # Create missing folders
        created = []
        for folder in selected_folders:
            full_path = os.path.join(temp_dir, folder)
            os.makedirs(full_path, exist_ok=True)
            created.append(folder)

        return f"Successfully created folders: {', '.join(created)}"

    except Exception as e:
        return f"Error: {str(e)}"
        print(f"Error encountered: {e}")
        raise  # Re-raise to let Gradio display the error
    finally:
        # Cleanup temp_dir
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

# Gradio Interface
interface = gr.Interface(
    fn=handle_user_actions,
    inputs=gr.File(label="Upload folder structure as .zip"),
    outputs=[
        gr.Textbox(label="Compliance Result"),
        gr.Dataframe(label="Folder Structure"),
        gr.Dataframe(label="Missing Folders"),
        gr.Textbox(label="Instructions"),
        gr.Textbox(label="Standard Folder Structure"),
        gr.Textbox(label="Debug Logs"),  # Add a new field for logs
    ],
)

# Launch the interface
interface.launch(share=True)

Paradigm Array: ['Folder1', 'Folder2', 'Folder3/Subfolder']
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2e3b36b575de836bd2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
#Cell 7 -
import logging
import gradio as gr
import os
import shutil
import zipfile

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Display the paradigm array as the "Standard Folder Structure"
def standard_folder_view():
    return paradigm_array.tolist()

# Compare folder structures and handle user input
def handle_user_actions_with_standard(zip_file):
    result, result_table, missing_table = compare_folder_structures(zip_file)

    # If the folder structure is compliant
    if result == "Standards Compliant":
        return (
            result,
            result_table,
            None,
            paradigm_array.tolist(),
            "Folder structure matches the standard.",
        )

    # For non-compliant structures, show missing folders and instructions
    missing_folders_checkboxes = [
        {"Label": folder["Missing Folder"], "Value": False} for folder in missing_table
    ]

    return (
        result,
        result_table,
        missing_folders_checkboxes,
        paradigm_array.tolist(),
        "Check the boxes for missing folders to add, then click 'Proceed' or 'Cancel'.",
    )

# Folder creation logic based on user input
def create_missing_folders(zip_file, selected_folders):
    temp_dir = "temp_dir"
    try:
        # Re-extract zip file to a temporary directory
        os.makedirs(temp_dir, exist_ok=True)
        with zipfile.ZipFile(zip_file.name, 'r') as z:
            z.extractall(temp_dir)

        # Create missing folders
        created_folders = []
        for folder in selected_folders:
            if folder["Value"]:  # If checkbox is checked
                full_path = os.path.join(temp_dir, folder["Label"])
                os.makedirs(full_path, exist_ok=True)
                created_folders.append(full_path)

        return f"Created missing folders: {', '.join(created_folders)}"
    except Exception as e:
        return f"Error: {str(e)}"
    finally:
        # Clean up temporary directory
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)

# Add buttons for interactivity
def action_buttons(selected_folders):
    return f"Selected actions: {', '.join([folder['Label'] for folder in selected_folders if folder['Value']])}"

# Gradio Interface
interface = gr.Interface(
    fn=handle_user_actions_with_standard,
    inputs=gr.File(label="Upload folder structure as .zip"),
    outputs=[
        gr.Textbox(label="Compliance Result"),
        gr.Dataframe(label="Folder Structure"),
        gr.CheckboxGroup(label="Missing Folders to Add"),
        gr.Dataframe(label="Standard Folder Structure"),
        gr.Textbox(label="Instructions"),
    ],
    live=True,
)

# Log the launch of the interface
logging.info("Launching Gradio interface...")

# Proceed and Cancel Button Actions
def proceed_action(zip_file, selected_folders):
    return create_missing_folders(zip_file, selected_folders)

def cancel_action():
    return "Operation canceled. No changes were made."

# Launch the Interface
try:
    interface.launch(share=True)
    logging.info("Gradio interface launched successfully.")
except Exception as e:
    logging.error(f"Error while launching Gradio interface: {e}")

print("Debugging test: Is this visible?")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c92e9a9f42ffe88dca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Debugging test: Is this visible?


In [ ]:
#interface.launch(debug=True, share=true)

In [ ]:
print("Testing output visibility in Colab.")